<a href="https://colab.research.google.com/github/susansunshin/Project-2/blob/main/Proejct_2_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 2 Part 2

Susan Shin

##Load

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (roc_auc_score, ConfusionMatrixDisplay, 
                             PrecisionRecallDisplay, RocCurveDisplay, 
                             f1_score, accuracy_score, classification_report)
from sklearn import set_config
set_config(display='diagram')

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [28]:
from google.colab import drive
drive.mount('/content/drive')

filename = '/content/sample_data/wines_SPA 3.csv'
df = pd.read_csv(filename)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


##Explore Data

In [29]:
#Make Copy
df_ml = df

In [30]:
#Remove num_reviews
df_ml = df_ml.drop(columns=["num_reviews"])
df_ml.head()

,winery,wine,year,rating,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [31]:
#Check for Dupes
df_ml.duplicated().sum()

5452

In [32]:
#Drop Dupes
df_ml = df_ml.drop_duplicates()
df_ml.duplicated().sum()

0

In [33]:
#Check for NAs
df_ml.isna().sum()

winery       0
wine         0
year         2
rating       0
country      0
region       0
price        0
type       106
body       271
acidity    271
dtype: int64

In [34]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2048 entries, 0 to 6100
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   winery   2048 non-null   object 
 1   wine     2048 non-null   object 
 2   year     2046 non-null   object 
 3   rating   2048 non-null   float64
 4   country  2048 non-null   object 
 5   region   2048 non-null   object 
 6   price    2048 non-null   float64
 7   type     1942 non-null   object 
 8   body     1777 non-null   float64
 9   acidity  1777 non-null   float64
dtypes: float64(4), object(6)
memory usage: 176.0+ KB


In [35]:
#Check for consistencies for object values
pd.set_option('display.max_column', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', -1)
pd.set_option('display.max_rows', None)

dtypes = df_ml.dtypes
str_cols = dtypes[dtypes=='object'].index
str_cols
  
for col in str_cols:
  print(f"- Column= {col}")
  print(df[col].value_counts(dropna=False))
  print('\n\n')

- Column= winery
Contino                                             457
Artadi                                              261
La Rioja Alta                                       254
Sierra Cantabria                                    237
Matarromera                                         232
Vina Pedrosa                                        230
Imperial                                            228
Losada                                              227
Portal del Priorat                                  225
Ramon Bilbao                                        225
Sei Solo                                            225
Matsu                                               224
Bodegas La Horra                                    223
Campillo                                            223
Lustau                                              223
La Vicalanda                                        223
Conreria d'Scala Dei                                223
Mar de Frades                  

The object in year is N.V. - which means Non Vintage. So this is a valid value.

Country is all Spain, no need for this column, will delete.

In [36]:
#Remove num_reviews
df_ml = df_ml.drop(columns=["country"])
df_ml.head()

,winery,wine,year,rating,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [37]:
#change year to object
df_ml["year"] = df_ml["year"].astype(object)

In [38]:
#Check Numerical values for outliers
df_ml["rating"].value_counts()

4.3    706
4.4    484
4.5    281
4.2    228
4.6    191
4.7    112
4.8     44
4.9      2
Name: rating, dtype: int64

In [39]:
df_ml["body"].value_counts()

4.0    1003
5.0     634
3.0     106
2.0      34
Name: body, dtype: int64

In [40]:
df_ml["acidity"].value_counts()

3.0    1672
2.0      70
1.0      35
Name: acidity, dtype: int64

Data is now ready for preprocessing!

##Preprocessing

In [41]:
#Split Data
X = df_ml.drop(columns=["type"])
y = df_ml["type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [42]:
X_train.isna().sum()

winery       0
wine         0
year         2
rating       0
region       0
price        0
body       207
acidity    207
dtype: int64

In [43]:
#Create selectors
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

In [44]:
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')

In [49]:
#Create pipes
num_pipe = make_pipeline(mean_imputer, scaler)
cat_pipe = make_pipeline(freq_imputer, ohe)

In [50]:
#Tupes
num_tup = (num_pipe, num_selector)
cat_tup = (cat_pipe, cat_selector)

In [51]:
#Create preprocessor
preprocessor = make_column_transformer(num_tup, cat_tup, remainder = 'passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb9480ffa90>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb9480ffb90>)])

##Logistic Regression Model

In [52]:
#Create pipeline and fit
logreg = LogisticRegression(random_state = 42)
logreg_pipe = make_pipeline(preprocessor, logreg)
logreg_pipe.fit(X_train,y_train)

ValueError: ignored

In [ ]:
#View scores
print(logreg_pipe.score(X_train, y_train))
print(logreg_pipe.score(X_test, y_test))

Without tuning this model has a test score of 91.5%

###Tune

In [ ]:
#Check Logreg parameters
logreg_pipe.get_params()

In [ ]:
#Tune C
logreg_param_grid = {"logisticregression__C": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}

logreg_pipe_gs = GridSearchCV(logreg_pipe, logreg_param_grid)
logreg_pipe_gs.fit(X_train, y_train)
print('Best Parameters:')
print(logreg_pipe_gs.best_params_)
best_pipe = logreg_pipe_gs.best_estimator_
print(f'Accuracy of best model is: {best_pipe.score(X_test, y_test)}')

In [ ]:
#Create tuned pipe
logreg_tuned = LogisticRegression(random_state = 42, C = 10)
logreg_pipe_tuned = make_pipeline(preprocessor, logreg_tuned)
logreg_pipe_tuned.fit(X_train,y_train)

###Score

In [ ]:
#Define loop for visualization
def evaluate (lgr_pipe, X_test, y_test, label="Test Data"):
  test_preds = lgr_pipe.predict(X_test)


   ## Print classification report for the test data (and make it pretty!)
  print('---'*20)
  print(f"[i] CLASSIFICATION REPORT FOR: {label}")
  print('---'*20)
  # print the classiffication report from the test data
  print(classification_report(y_test, test_preds))

  fig, axes = plt.subplots(1,3, figsize=(15,5))

  # create a confusion matrix with the test data
  ConfusionMatrixDisplay.from_predictions(y_test, test_preds, 
               normalize='true', cmap='Greens',
               ax = axes[0]);
  ConfusionMatrixDisplay.from_predictions(y_test, test_preds, 
               cmap='Greens',
               ax = axes[1]);

  RocCurveDisplay.from_estimator(lgr_pipe, X_test, y_test, ax=axes[2])
  plt.show()

In [ ]:
evaluate(logreg_pipe, X_test, y_test, label='Test Data')

##Random Forest Model

In [ ]:
rf = RandomForestClassifier(random_state = 42)
rf_pipe = make_pipeline(preprocessor, rf)
rf_pipe.fit(X_train,y_train)

In [ ]:
print(rf_pipe.score(X_train, y_train))
print(rf_pipe.score(X_test, y_test))

###Tune

In [ ]:
#Get Parameters
rf_pipe.get_params()

In [ ]:
#Find max depth
est_depths = [estimator.get_depth() for estimator in rf.estimators_]
max(est_depths)

In [ ]:
#Run grid search
rf_param_grid = {'randomforestclassifier__max_depth': range(1,14), 
                      'randomforestclassifier__min_samples_leaf': [3,15,20],
                      'randomforestclassifier__min_samples_split': [2, 10, 100]}

rf_pipe_gs = GridSearchCV(rf_pipe, rf_param_grid)
rf_pipe_gs.fit(X_train, y_train)
print('Best Parameters:')
print(rf_pipe_gs.best_params_)
best_pipe = rf_pipe_gs.best_estimator_
print(f'Accuracy of best model is: {best_pipe.score(X_test, y_test)}')

In [ ]:
#Create tuned pipe
rf_tuned = RandomForestClassifier(max_depth = 1, min_samples_leaf = 3, min_samples_split = 2, random_state = 42)
rf_pipe_tuned = make_pipeline(preprocessor, rf)
rf_pipe_tuned.fit(X_train,y_train)

###Score

In [ ]:
evaluate(rf_pipe_tuned, X_test, y_test, label='Test Data')

##KNeighbors Model

In [ ]:
#Create Pipe
kn = KNeighborsClassifier()
kn_pipe = make_pipeline(preprocessor, kn)
kn_pipe.fit(X_train,y_train)

In [ ]:
#Check score
print(kn_pipe.score(X_train, y_train))
print(kn_pipe.score(X_test, y_test))

###Tune

In [ ]:
#Get Parameters
kn_pipe.get_params()

In [ ]:
#Run grid search
kn_param_grid = {"kneighborsclassifier__n_neighbors": range(1,50)}

kn_pipe_gs = GridSearchCV(kn_pipe, kn_param_grid)
kn_pipe_gs.fit(X_train, y_train)
print('Best Parameters:')
print(kn_pipe_gs.best_params_)
best_pipe = kn_pipe_gs.best_estimator_
print(f'Accuracy of best model is: {best_pipe.score(X_test, y_test)}')

In [ ]:
#Create tuned model
kn_tuned = KNeighborsClassifier(n_neighbors = 1)
kn_pipe_tuned = make_pipeline(preprocessor, kn_tuned)
kn_pipe_tuned.fit(X_train,y_train)

In [ ]:
print(kn_pipe_tuned.score(X_train, y_train))
print(kn_pipe_tuned.score(X_test, y_test))

###Score

In [ ]:
evaluate(kn_pipe_tuned, X_test, y_test, label='Test Data')

#Conclusion

In [ ]:
#All three scores together
evaluate(logreg_pipe_tuned, X_test, y_test, label='Logistic Regression Test Data')
evaluate(rf_pipe_tuned, X_test, y_test, label='Random Forest Test Data')
evaluate(kn_pipe_tuned, X_test, y_test, label='KNeighbors Test Data')